In [ ]:
import modulos as md
import numpy as np
import os
import sys
import constantes as cons
lista_filtrada = []
lista=md.filtradodemonedas ()
#lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'KNCUSDT', 'DOGEUSDT', 'WAVESUSDT', 'MKRUSDT', 'DOTUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SOLUSDT', 'STORJUSDT', 'BLZUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT', 'TOMOUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'MATICUSDT', 'UNFIUSDT', 'LINAUSDT', '1000SHIBUSDT', 'MASKUSDT', 'DYDXUSDT', 'GALAUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'LDOUSDT', 'APTUSDT', 'FETUSDT', 'MINAUSDT', 'CFXUSDT', 'STXUSDT', 'ARBUSDT', 'BLURUSDT', 'SUIUSDT', '1000PEPEUSDT', 'WLDUSDT', 'AGLDUSDT', 'YGGUSDT', 'CYBERUSDT', 'STRAXUSDT', 'LOOMUSDT', 'BIGTIMEUSDT', 'ORBSUSDT', 'POLYXUSDT']
#lista = ['INJUSDT']

timeframe='15m'
if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
print(lista)
porcentajes_sumados=0
win_rate_buenos = 0
win_rate_malos = 0
trades = 0
for symbol in lista:    
    try:
        #print(symbol)
        data = md.estrategia_smart(symbol, debug = False, refinado = False, file_source = False, timeframe = timeframe)
        resultado=md.backtesting_smart(data, plot_flag=imprimo, symbol=symbol)
        if resultado['Win Rate [%]'] >= 50:
            lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
        porcentajes_sumados=porcentajes_sumados+resultado['Return [%]']
        trades = trades + resultado['# Trades']
        if resultado['Win Rate [%]'] >= 50:
            win_rate_buenos = win_rate_buenos+1
        else:
            win_rate_malos = win_rate_malos+1
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
print(f"\nTimeframe {timeframe} - porcentajes_sumados {md.truncate(porcentajes_sumados,2)} - trades {trades} - win_rate_buenos {win_rate_buenos} - win_rate_malos {win_rate_malos}")
#(data.loc[data['Open Time'] == '2023-10-03 03:00:00'])
